# Identifying Ethnicity in OpenSAFELY-TPP

This short report describes how ethnicity can be identified in the OpenSAFELY-TPP database, and the strengths and weaknesses of the methods. Ethnicity is known to be an important determinant of health outcomes, particularly during the COVID-19 outbreak where a complex interplay of social and biological factors resulted in increased exposure, reduced protection, and increased severity of illness. The recording of patients' ethnic group in primary care can support efforts to achieve equity in service provision and outcomes. 

The [NHS Data Model and Dictionary](https://www.datadictionary.nhs.uk/data_elements/ethnic_category.html?hl=ethnicity) states that ethnic data groups defined in the [2001 census](https://www.ethnicity-facts-figures.service.gov.uk/style-guide/ethnic-groups#2001-census) is the national mandatory standard for the collection and analysis of ethnicity.

In OpenSAFELY-TPP, there is no categorical "ethnicity" variable to record this information. Rather, ethnicity is recorded using clinical codes, like any other clinical or administrative event, with specific codes relating to specific ethnic groups. 

We define three codelists to capture primary care ethnicity in OpenSAFELY-TPP :  One Clinical Terms Version 3 (CTV3) Codelist "[CTV3:2020](https://www.opencodelists.org/codelist/opensafely/ethnicity/2020-04-27)", and two systematised nomenclature of medicine clinical terms (SNOMED CT) codelists "[SNOMED:2022](https://www.opencodelists.org/codelist/opensafely/ethnicity-snomed-0removed/2e641f61/)" and "[PRIMIS:2021](https://www.opencodelists.org/codelist/primis-covid19-vacc-uptake/eth2001/v1/)".

It is common for OpenSAFELY studies to supplement the primary care recorded ethnicity, where missing, with ethnicity data from the Secondary Uses Service (SUS). This report has focussed solely on the primary care recorded ethnicity. The representativeness of the CTV3:2020 coded ethnicity supplemented with SUS data has been [reported previously](https://wellcomeopenresearch.org/articles/7-191/v1).

This is a living document that will be updated to reflect changes to the OpenSAFELY-TPP database and the patient records within.

## OpenSAFELY
OpenSAFELY is an analytics platform for conducting analyses on Electronic Health Records inside the secure environment where the records are held. This has multiple benefits: 

* We don't transport large volumes of potentially disclosive pseudonymised patient data outside of the secure environments for analysis
* Analyses can run in near real-time as records are ready for analysis as soon as they appear in the secure environment
* All infrastructure and analysis code is stored in GitHub repositories, which are open for security review, scientific review, and re-use

A key feature of OpenSAFELY is the use of study definitions, which are formal specifications of the datasets to be generated from the OpenSAFELY database. This takes care of much of the complex Electronic Healthcare Records (EHR) data wrangling required to create a dataset in an analysis-ready format. It also creates a library of standardised and validated variable definitions that can be deployed consistently across multiple projects. 

The purpose of this report is to describe the main variables that relate ethnicity, and their relative strengths and weaknesses.

## Available Records
OpenSAFELY-TPP runs inside TPP's data centre which contains the primary care records for all patients registered at practices using TPP's SystmOne Clinical Information System. This data centre also imports external datasets from other sources, including A&E attendances and hospital admissions from NHS Digital's Secondary Use Service, and death registrations from the ONS. More information on available data sources can be found within the [OpenSAFELY documentation](https://docs.opensafely.org/data-sources/intro/). 

# Methods

We define three codelists to capture primary care ethnicity in OpenSAFELY-TPP : "[CTV3:2020](https://www.opencodelists.org/codelist/opensafely/ethnicity/2020-04-27)", "[SNOMED:2022](https://www.opencodelists.org/codelist/opensafely/ethnicity-snomed-0removed/2e641f61/)" and "[PRIMIS:2021](https://www.opencodelists.org/codelist/primis-covid19-vacc-uptake/eth2001/v1/)".



### [Completeness of ethnicity data](#complete_cell)
To evaluate how well each of these codelists are populated, the proportion of patients with ethnicity recorded (that is, the presence of any code in the codelist in the patient record) was calculated for patients registered as of 1 January 2022. 

We examine trends across the whole population and by each of the following demographic and clinical subgroups to detect any inequalities.

Demographic covariates:

- Age band
- Sex
- Ethnicity
- Region
- IMD


Clinical covariates:

- Dementia
- Diabetes
- Learning disability


### [Ethnicity by group](#group_cell)

These codes were grouped into one of two ethnicity groups based on the 2001 Census groups: 

5-level group: 
- Asian or Asian British
- Black or Black British 
- Mixed 
- White 
- Chinese or other ethnic group 



16-level group: 
- Asian or Asian British
    - Indian
    - Pakistani
    - Bangladeshi
    - Any other Asian background
- Black or Black British 
    - Caribbean
    - African
    - Any other Black background
- Mixed 
    - White and Black Caribbean
    - White and Black African
    - White and Asian
    - Any other Mixed background
- White 
    - British
    - Irish
    - Any other White background
- Chinese or other ethnic group 
    - Chinese
    - Any other
    

For patients with multiple ethnicity records, the most recent record was chosen (even if this is later than the cohort date). The proportion of patients with each ethnicity groups was calculated, within each clinical and demographic subgroup.


### [Changes in coded ethnicity groups](#changes_cell) 
In order to investigate the extent of discrepancies within individual patients' recorded grouped ethnicity the proportion of patients with any grouped ethnicity recorded which does not match their 'latest' recorded grouped ethnicity was calculated for each of the five ethnic groups. 

### [Comparison of 'Latest' and 'Most Frequent' coded ethnicity](#frequent_cell)

The proportion of patients with a recorded latest ethnicity whose most frequently recorded ethnicity does not match their latest recorded ethnicity was calculated for each of the five ethnic groups.

All patient counts are rounded to the nearest 5. Percentages may not add to exactly 100 due to rounding.


In [356]:
import sys

In [357]:
import os
import pandas as pd
import numpy as np
from itertools import product
from IPython.display import display, Markdown, Image
from datetime import date, timedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.options.mode.chained_assignment = None 
pd.options.display.float_format = '{:,.0f}'.format

In [358]:

def local_patient_counts(
    definitions, output_path, code_dict="", categories=False, missing=False,quietly =False
):
    import pandas as pd

    suffix = "_filled"
    overlap = "all_filled"
    if missing == True:
        suffix = "_missing"
        overlap = "all_missing"
    if categories:
        ### uncomment to switch to proportion of recorded ethnicity instead of prop of TPP population
        # df_population = pd.read_csv(
        #     f"../output/{output_path}/simple_patient_counts_registered.csv"
        # ).set_index(["group", "subgroup"])
        

        df_append = pd.read_csv(
            f"../output/{output_path}/simple_patient_counts_categories_registered.csv"
        ).set_index(["group", "subgroup"])
        
        if output_path == output_path_5:
            global df_append_cat_5
            df_append_cat_5 = df_append

        if output_path == output_path_16:
            global df_append_cat_16
            df_append_cat_16 = df_append
        ### uncomment to switch to proportion of recorded ethnicity instead of prop of TPP population
        # df_append.drop("population", inplace=True, axis=1)
        for definition in definitions:
            ### uncomment to switch to proportion of recorded ethnicity instead of prop of TPP population
            # df_append[f"population_{definition}"] = df_population[definition+"_filled"]
        # ensure definitions[n] in code_dict[definitions[n]] below refers to one of the definitions of interest
            full_definitions = [
                f"{category}_{definition}"
                for category, definition in product(
                    code_dict[definitions[1]].values(), [definition]
                )
            ]
            for full_definition in full_definitions:
                if missing:
                    
                    df_append[full_definition + suffix] = (
                        ### uncomment to switch to proportion of recorded ethnicity instead of prop of TPP population
                        # df_append[f"population_{definition}"] - df_append[full_definition + "_filled"]
                        df_append[f"population"] - df_append[full_definition + "_filled"]
                    )  
                ## uncomment to switch to proportion of recorded ethnicity instead of prop of TPP population    
                df_append[full_definition + "_pct"] = round(
                    (df_append[full_definition + suffix].div(df_append[f"population"])) * 100, 1
                )
                df_append[overlap + "_pct"] = round(
                    (df_append[overlap].div(df_append[f"population"])) * 100, 1
                )

                # Combine count and percentage columns
                df_append[full_definition] = (
                    df_append[full_definition + suffix].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_append[full_definition + "_pct"].astype(str)
                    + ")"
                )
                df_append = df_append.drop(columns=[full_definition + suffix, full_definition + "_pct"])
    else:
        df_append = pd.read_csv(
            f"../output/{output_path}/simple_patient_counts_registered.csv"
        ).set_index(["group", "subgroup"])
        global total
        total =  df_append
        for definition in definitions:
            if missing:
                df_append[definition + suffix] = (
                    df_append["population"] - df_append[definition + "_filled"]
                )    
            df_append[definition + "_pct"] = round(
                (df_append[definition + suffix].div(df_append["population"])) * 100, 1
            )
            df_append[overlap + "_pct"] = round(
                (df_append[overlap].div(df_append["population"])) * 100, 1
            )

            # Combine count and percentage columns
            df_append[definition] = (
                df_append[definition + suffix].apply(lambda x: "{:,.0f}".format(x))
                + " ("
                + df_append[definition + "_pct"].astype(str)
                + ")"
            )
            df_append = df_append.drop(columns=[definition + suffix, definition + "_pct"])
    df_append[overlap] = (
        df_append[overlap].apply(lambda x: "{:,.0f}".format(x))
        + " ("
        + df_append[overlap + "_pct"].astype(str)
        + ")"
    )
    df_append = df_append.reset_index()
    df_append = df_append.replace("True", "Yes")
    df_append = df_append.replace("False", "No")
    df_append = df_append.set_index(["group", "subgroup"])
    df_append = df_append.drop(columns=[overlap + "_pct"])
    if categories:
        full_definitions = [
                f"{category}_{definition}"
                for category, definition in product(
                    code_dict[definitions[1]].values(), definitions
                )
            ]
        df_patient_counts = df_append[full_definitions]
    else: 
        df_patient_counts = df_append[definitions + [overlap] + ["population"]]
    # Final redaction step
    df_patient_counts = df_patient_counts.replace(np.nan, "-")
    df_patient_counts = df_patient_counts.replace("nan (nan)", "- (-)")
    for k, v in definition_dict.items():
        df_patient_counts.columns = df_patient_counts.columns.str.replace(k,v) 
    df_patient_counts.columns = df_patient_counts.columns.str.replace("_", " ")
    if quietly == False:
        display(df_patient_counts)
        print(" \n")
    
    if categories:
        df_patient_counts.to_csv(
                f"../output/{output_path}/local_patient_counts_categories_registered.csv"
            )
    
    

In [359]:
### CONFIGURE ###
definitions_5 = [ 'ethnicity_new_5','ethnicity_5', 'ethnicity_primis_5']
definitions_16 = ['ethnicity_new_16', 'ethnicity_16', 'ethnicity_primis_16']
covariates = ['_age_band','_sex','_region','_imd','_dementia','_diabetes','_hypertension','_learning_disability']
output_path_5 = 'simplified_output/5_group/tables'
output_path_16 = 'simplified_output/16_group/tables'
suffixes = ['','_missing']
suffix = ''
code_dict_5 = {
    "imd": {
        0: "Unknown",
        1: "1 Most deprived",
        2: "2",
        3: "3",
        4: "4",
        5: "5 Least deprived",
    },
    "ethnicity_5": {1: "Asian", 2: "Black", 3: "Mixed", 4: "White", 5: "Other"},
    "ethnicity_new_5": {1: "Asian", 2: "Black", 3: "Mixed", 4: "White", 5: "Other"},
    "ethnicity_primis_5": {1: "Asian", 2: "Black", 3: "Mixed", 4: "White", 5: "Other"},
}

# Code dictionary
code_dict_16 = {
    "imd": {
        0: "Unknown",
        1: "1 Most deprived",
        2: "2",
        3: "3",
        4: "4",
        5: "5 Least deprived",
    },
    "ethnicity_16": {
        1: "Indian",
        2: "Pakistani",
        3: "Bangladeshi",
        4: "Other_Asian",
        5: "Caribbean",
        6: "African",
        7: "Other_Black",
        8: "White_and_Black_Caribbean",
        9: "White_and_Black_African",
        10: "White_and_Asian",
        11: "Other_Mixed",
        12: "White_British",
        13: "White_Irish",
        14: "Other_White",
        15: "Chinese",
        16: "Any_other_ethnic_group",
    },
    "ethnicity_new_16": {
        1: "Indian",
        2: "Pakistani",
        3: "Bangladeshi",
        4: "Other_Asian",
        5: "Caribbean",
        6: "African",
        7: "Other_Black",
        8: "White_and_Black_Caribbean",
        9: "White_and_Black_African",
        10: "White_and_Asian",
        11: "Other_Mixed",
        12: "White_British",
        13: "White_Irish",
        14: "Other_White",
        15: "Chinese",
        16: "Any_other_ethnic_group",
    },
    "ethnicity_primis_16": {
        1: "Indian",
        2: "Pakistani",
        3: "Bangladeshi",
        4: "Other_Asian",
        5: "Caribbean",
        6: "African",
        7: "Other_Black",
        8: "White_and_Black_Caribbean",
        9: "White_and_Black_African",
        10: "White_and_Asian",
        11: "Other_Mixed",
        12: "White_British",
        13: "White_Irish",
        14: "Other_White",
        15: "Chinese",
        16: "Any_other_ethnic_group",
    },
}

definition_dict = {
        "ethnicity_new_5": "5 SNOMED:2022",
        "ethnicity_primis_5": "5 PRIMIS:2021",
        "ethnicity_5": "5 CTV3:2020",
        "ethnicity_new_16": "16 SNOMED:2022",
        "ethnicity_primis_16": "16 PRIMIS:2021",
        "ethnicity_16": "16 CTV3:2020",
}


In [360]:
# get data extraction date
extract_date = pd.to_datetime(os.path.getmtime(f"../output/{output_path_5}/simple_patient_counts_registered.csv"), unit='s')
# get notebook run date
run_date = date.today()

## Results

<a id='complete_cell'></a> 

### Completeness of ethnicity data 

In [361]:
### Run quietly to obtain globals for Markdown
local_patient_counts(
         definitions_5,  output_path_5,quietly=True
    )


In [362]:
display(Markdown(f"""
Around {float('%.3g' % total["all_filled"][0])/1000000} million patients who have been registered in OpenSAFELY-TPP are identified in all three codelists. CTV3:2020 is the most well-populated with {float('%.3g' % total["ethnicity_new_5_filled"][0])/1000000} million patients having at least one CTV3:2020 recording of ethnicity. 
"""))


Around 0.000255 million patients who have been registered in OpenSAFELY-TPP are identified in all three codelists. CTV3:2020 is the most well-populated with 0.00048 million patients having at least one CTV3:2020 recording of ethnicity. 


In [363]:
local_patient_counts(
         definitions_5,  output_path_5
    )

5 SNOMED:2022 5 CTV3:2020  \
group               subgroup                                             
all                 with records                480 (73.8)  490 (75.4)   
age_band            0-19                         45 (75.0)   40 (66.7)   
                    20-29                        75 (75.0)   70 (70.0)   
                    30-39                        65 (68.4)   70 (73.7)   
                    40-49                        60 (70.6)   65 (76.5)   
                    50-59                        55 (73.3)   60 (80.0)   
                    60-69                        70 (82.4)   65 (76.5)   
                    70-79                        55 (73.3)   50 (66.7)   
                    80+                          60 (80.0)   65 (86.7)   
sex                 F                           235 (75.8)  225 (72.6)   
                    M                           250 (73.5)  265 (77.9)   
region              East Midlands                30 (66.7)   35 (77.8)   
                    East of England              25 (62.5)   30 (75.0)   
                    London                       65 (81.2)   60 (75.0)   
                    North East                   30 (75.0)   30 (75.0)   
                    North West                   20 (57.1)   30 (85.7)   
                    South East                   35 (77.8)   30 (66.7)   
                    South West                   45 (81.8)   40 (72.7)   
                    West Midlands                30 (75.0)   30 (75.0)   
                    Yorkshire and the Humber     35 (77.8)   30 (66.7)   
imd                 1 Most deprived              90 (72.0)   90 (72.0)   
                    2                           105 (77.8)  110 (81.5)   
                    3                            85 (73.9)   80 (69.6)   
                    4                            95 (76.0)  100 (80.0)   
                    5 Least deprived            105 (72.4)  110 (75.9)   
                    Unknown                          - (-)       - (-)   
dementia            No                          475 (73.6)  490 (76.0)   
                    Yes                              - (-)       - (-)   
diabetes            No                          475 (74.2)  485 (75.8)   
                    Yes                              - (-)       - (-)   
hypertension        No                          475 (73.6)  485 (75.2)   
                    Yes                              - (-)       - (-)   
learning_disability No                          475 (73.6)  485 (75.2)   
                    Yes                              - (-)       - (-)   

                                             5 PRIMIS:2021  all filled  \
group               subgroup                                             
all                 with records                475 (73.1)  255 (39.2)   
age_band            0-19                         50 (83.3)   25 (41.7)   
                    20-29                        75 (75.0)   40 (40.0)   
                    30-39                        65 (68.4)   30 (31.6)   
                    40-49                        55 (64.7)   25 (29.4)   
                    50-59                        50 (66.7)   30 (40.0)   
                    60-69                        70 (82.4)   45 (52.9)   
                    70-79                        55 (73.3)   25 (33.3)   
                    80+                          60 (80.0)   35 (46.7)   
sex                 F                           235 (75.8)  115 (37.1)   
                    M                           245 (72.1)  140 (41.2)   
region              East Midlands                35 (77.8)   15 (33.3)   
                    East of England              25 (62.5)   15 (37.5)   
                    London                       60 (75.0)   35 (43.8)   
                    North East                   30 (75.0)   15 (37.5)   
                    North West                   30 (85.7)   10 (28.6)   
                    South East                  

<a id='group_cell'></a> 

### Ethnicity by group

#### 5 Group

In [364]:
### Run quietly to obtain globals for Markdown
local_patient_counts(
         definitions_5,  output_path_5,code_dict_5, categories=True,missing=False,quietly=True
    )

In [365]:
display(Markdown(f"""
The SNOMED:2022 is the most well-populated codelist for `White` ({'{:,.0f}'.format(float('{:,.3g}'.format(df_append_cat_5["White_ethnicity_new_5_filled"][0])))}), `Mixed` ({'{:,.0f}'.format(float('{:,.3g}'.format(df_append_cat_5["Mixed_ethnicity_new_5_filled"][0])))}), `Asian` ({'{:,.0f}'.format(float('{:,.3g}'.format(df_append_cat_5["Asian_ethnicity_new_5_filled"][0])))}) and `Black` ({'{:,.0f}'.format(float('{:,.3g}'.format(df_append_cat_5["Black_ethnicity_new_5_filled"][0])))}) ethnicities. The CTV3:2020 codelist classifies more people as `Other` than the SNOMED:2022 codelist ({'{:,.0f}'.format(float('{:,.3g}'.format(df_append_cat_5["Other_ethnicity_5_filled"][0])))} and {'{:,.0f}'.format(float('{:,.3g}'.format(df_append_cat_5["Other_ethnicity_new_5_filled"][0])))} respectively), however, the CTV3:2020 codelist includes some codes relating to religion rather than ethnicity (e.g. "XaJSe: Muslim - ethnic category 2001 census") which were excluded from the SNOMED:2022 codelist.
"""))



The SNOMED:2022 is the most well-populated codelist for `White` (285), `Mixed` (55), `Asian` (45) and `Black` (50) ethnicities. The CTV3:2020 codelist classifies more people as `Other` than the SNOMED:2022 codelist (115 and 50 respectively), however, the CTV3:2020 codelist includes some codes relating to religion rather than ethnicity (e.g. "XaJSe: Muslim - ethnic category 2001 census") which were excluded from the SNOMED:2022 codelist.


In [366]:
display(Markdown(f"### Ethnicity by 5 group"))

### Ethnicity by 5 group

In [367]:
local_patient_counts(
         definitions_5,  output_path_5,code_dict_5, categories=True,missing=False
    )

Asian 5 SNOMED:2022  \
group               subgroup                                       
all                 with records                        45 (6.9)   
age_band            0-19                                   - (-)   
                    20-29                              10 (10.0)   
                    30-39                                  - (-)   
                    40-49                                  - (-)   
                    50-59                                  - (-)   
                    60-69                              10 (11.8)   
                    70-79                                  - (-)   
                    80+                                10 (13.3)   
sex                 F                                   25 (8.1)   
                    M                                   25 (7.4)   
region              East Midlands                          - (-)   
                    East of England                        - (-)   
                    London                                 - (-)   
                    North East                             - (-)   
                    North West                             - (-)   
                    South East                             - (-)   
                    South West                             - (-)   
                    West Midlands                          - (-)   
                    Yorkshire and the Humber               - (-)   
imd                 1 Most deprived                     10 (8.0)   
                    2                                  15 (11.1)   
                    3                                   10 (8.7)   
                    4                                      - (-)   
                    5 Least deprived                    10 (6.9)   
                    Unknown                                - (-)   
dementia            No                                  45 (7.0)   
                    Yes                                    - (-)   
diabetes            No                                  45 (7.0)   
                    Yes                                    - (-)   
hypertension        No                                  45 (7.0)   
                    Yes                                    - (-)   
learning_disability No                                  45 (7.0)   
                    Yes                                    - (-)   

                                             Asian 5 CTV3:2020  \
group               subgroup                                     
all                 with records                    100 (15.4)   
age_band            0-19                             15 (25.0)   
                    20-29                            10 (10.0)   
                    30-39                            20 (21.1)   
                    40-49                            20 (23.5)   
                    50-59                            10 (13.3)   
                    60-69                            10 (11.8)   
                    70-79                            10 (13.3)   
                    80+                              10 (13.3)   
sex                 F                                45 (14.5)   
                    M                                50 (14.7)   
region              East Midlands                        - (-)   
                    East of England                      - (-)   
                    London                           10 (12.5)   
                    North East                       10 (25.0)   
                    North West                           - (-)   
                    South East                       10 (22.2)   
                    South West                       10 (18.2)   
                    West Midlands                        - (-)   
                    Yorkshire and the Humber             - (-)   
imd                 1 Most deprived                  15 (12.0)   
                    2                                20 (14.8)   
            

#### 16 Group

In [368]:
### Run quietly to obtain globals for Markdown
local_patient_counts(
         definitions_16,  output_path_16,code_dict_16, categories=True,missing=False,quietly=True
    )

In [369]:
display(Markdown(f"""
In the `16 group` ethnicity the `Other` ethnic group is expanded to `Chinese` and `Any other ethnic group`. For `Chinese` the SNOMED:2022 codelist is most well-populated  ({'{:,.0f}'.format(float('{:,.3g}'.format(df_append_cat_16["Chinese_ethnicity_new_16_filled"][0])))}) and for `Any other ethnic group` the CTV3:2020 codelist is most well populated ({'{:,.0f}'.format(float('{:,.3g}'.format(df_append_cat_16["Any_other_ethnic_group_ethnicity_16_filled"][0])))}) . 
"""))


In the `16 group` ethnicity the `Other` ethnic group is expanded to `Chinese` and `Any other ethnic group`. For `Chinese` the SNOMED:2022 codelist is most well-populated  (30) and for `Any other ethnic group` the CTV3:2020 codelist is most well populated (30) . 


In [370]:
display(Markdown(f"##### Ethnicity by 16 group"))

##### Ethnicity by 16 group

In [371]:
local_patient_counts(
         definitions_16,  output_path_16,code_dict_16, categories=True,missing=False
    )

Indian 16 SNOMED:2022  \
group               subgroup                                         
all                 with records                          25 (3.8)   
age_band            0-19                                     - (-)   
                    20-29                                    - (-)   
                    30-39                                    - (-)   
                    40-49                                    - (-)   
                    50-59                                    - (-)   
                    60-69                                    - (-)   
                    70-79                                    - (-)   
                    80+                                      - (-)   
sex                 F                                     10 (2.8)   
                    M                                     15 (5.1)   
region              East Midlands                            - (-)   
                    East of England                          - (-)   
                    London                                   - (-)   
                    North East                               - (-)   
                    North West                               - (-)   
                    South East                               - (-)   
                    South West                               - (-)   
                    West Midlands                            - (-)   
                    Yorkshire and the Humber                 - (-)   
imd                 1 Most deprived                          - (-)   
                    2                                        - (-)   
                    3                                        - (-)   
                    4                                     10 (7.1)   
                    5 Least deprived                         - (-)   
                    Unknown                                  - (-)   
dementia            No                                    25 (3.9)   
                    Yes                                      - (-)   
diabetes            No                                    25 (3.9)   
                    Yes                                      - (-)   
hypertension        No                                    25 (3.9)   
                    Yes                                      - (-)   
learning_disability No                                    25 (3.9)   
                    Yes                                      - (-)   

                                             Indian 16 CTV3:2020  \
group               subgroup                                       
all                 with records                        45 (6.9)   
age_band            0-19                               10 (15.4)   
                    20-29                                  - (-)   
                    30-39                                  - (-)   
                    40-49                                  - (-)   
                    50-59                                  - (-)   
                    60-69                                  - (-)   
                    70-79                                  - (-)   
                    80+                                    - (-)   
sex                 F                                   25 (7.0)   
                    M                                   20 (6.8)   
region              East Midlands                          - (-)   
                    East of England                        - (-)   
                    London                                 - (-)   
                    North East                             - (-)   
                    North West                             - (-)   
                    South East                             - (-)   
                    South West                             - (-)   
                    West Midlands                          - (-)   
                    Yorkshire and the Humber               - (-)   
imd                 1 Most d

<a id='changes_cell'></a> 

### Changes in coded ethnicity groups

#### 5 Group

In [372]:
for definition in definitions_5:
        lowerlist_5 = [x.lower() for x in (list(code_dict_5[definition].values()))]
        df_state_change = pd.read_csv(f'../output/{output_path_5}/simple_state_change_{definition}{suffix}_registered.csv').set_index(definition)
        df_state_change.columns = df_state_change.columns.str.replace(definition + "_", "")
        #resort rows
        df_state_change = df_state_change.reindex(list(code_dict_5[definition].values()))
        df_state_change = df_state_change.reset_index()
        
        df_state_change[definition]=df_state_change[definition]+": " +df_state_change["n"].apply(lambda x: "{:,.0f}".format(x))
        df_state_change = df_state_change.set_index(definition)
        globals()[f'df_sc_pct_{definition}'] = df_state_change
        for item in lowerlist_5 + list(["any"]):
            df_state_change[item + "_pct"]= round(
                    (df_state_change[item].div(df_state_change["n"])) * 100, 1
                )
        
            df_state_change[item] = (
                    df_state_change[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_state_change[item + "_pct"].astype(str)
                    + ")"
                )
        df_state_change=df_state_change[lowerlist_5 + list(["any"])]
        df_state_change = df_state_change.replace("nan (nan)", "- (-)")
        df_state_change = df_state_change.reset_index()
        df_state_change = df_state_change.rename(definition_dict, axis='columns')
        df_state_change.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_state_change = df_state_change.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        # display(df_state_change)

In [373]:
display(Markdown(f"""
Patients whose latest recorded ethnicity were grouped as `Mixed` were most likely to have a discordant ethnicity recording ({'{:,.0f}'.format(df_sc_pct_ethnicity_new_5["any_pct"][2])}% SNOMED:2022, {'{:,.0f}'.format(df_sc_pct_ethnicity_5["any_pct"][2])}% CTV3:2020, {'{:,.0f}'.format(df_sc_pct_ethnicity_primis_5["any_pct"][2])}% PRIMIS:2021). All three codelists had patients with latest recorded ethnicity grouped as `Black` who also had a recorded ethnicity of `White` ({'{:,.0f}'.format(df_sc_pct_ethnicity_new_5["white_pct"][1])}% SNOMED:2022, {'{:,.0f}'.format(df_sc_pct_ethnicity_5["white_pct"][1])}% CTV3:2020, {'{:,.0f}'.format(df_sc_pct_ethnicity_primis_5["white_pct"][1])}% PRIMIS:2021). 
"""))


Patients whose latest recorded ethnicity were grouped as `Mixed` were most likely to have a discordant ethnicity recording (64% SNOMED:2022, 62% CTV3:2020, 72% PRIMIS:2021). All three codelists had patients with latest recorded ethnicity grouped as `Black` who also had a recorded ethnicity of `White` (nan% SNOMED:2022, nan% CTV3:2020, 10% PRIMIS:2021). 


In [374]:
for definition in definitions_5:
        lowerlist_5 = [x.lower() for x in (list(code_dict_5[definition].values()))]
        df_state_change = pd.read_csv(f'../output/{output_path_5}/simple_state_change_{definition}{suffix}_registered.csv').set_index(definition)
        df_state_change.columns = df_state_change.columns.str.replace(definition + "_", "")
        #resort rows
        df_state_change = df_state_change.reindex(list(code_dict_5[definition].values()))
        df_state_change = df_state_change.reset_index()
        
        df_state_change[definition]=df_state_change[definition]+": " +df_state_change["n"].apply(lambda x: "{:,.0f}".format(x))
        df_state_change = df_state_change.set_index(definition)
        # globals()[f'df_sc_pct_{definition}'] = df_state_change
        for item in lowerlist_5 + list(["any"]):
            df_state_change[item + "_pct"]= round(
                    (df_state_change[item].div(df_state_change["n"])) * 100, 1
                )
        
            df_state_change[item] = (
                    df_state_change[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_state_change[item + "_pct"].astype(str)
                    + ")"
                )
        df_state_change=df_state_change[lowerlist_5 + list(["any"])]
        df_state_change = df_state_change.replace("nan (nan)", "- (-)")
        df_state_change = df_state_change.reset_index()
        df_state_change = df_state_change.rename(definition_dict, axis='columns')
        df_state_change.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n\n{definition_dict[definition]}'}, inplace=True)
        df_state_change = df_state_change.set_index(f'Latest Ethnicity-\n\n{definition_dict[definition]}')
        display(Markdown(f"##### Changes in {definition_dict[definition]} ethnicity"))
        display(df_state_change)
        print(" \n")

##### Changes in 5 SNOMED:2022 ethnicity

,asian,black,mixed,white,other,any
Latest Ethnicity- 5 SNOMED:2022,,,,,,
Asian: 45,- (-),- (-),- (-),- (-),- (-),30 (66.7)
Black: 50,- (-),- (-),- (-),- (-),- (-),30 (60.0)
Mixed: 55,- (-),- (-),- (-),- (-),- (-),35 (63.6)
White: 285,25 (8.8),30 (10.5),25 (8.8),25 (8.8),25 (8.8),205 (71.9)
Other: 50,- (-),- (-),- (-),- (-),- (-),35 (70.0)


##### Changes in 5 CTV3:2020 ethnicity

,asian,black,mixed,white,other,any
Latest Ethnicity- 5 CTV3:2020,,,,,,
Asian: 100,10 (10.0),10 (10.0),10 (10.0),10 (10.0),15 (15.0),65 (65.0)
Black: 85,10 (11.8),10 (11.8),10 (11.8),- (-),10 (11.8),55 (64.7)
Mixed: 105,15 (14.3),10 (9.5),10 (9.5),10 (9.5),10 (9.5),65 (61.9)
White: 85,- (-),10 (11.8),10 (11.8),10 (11.8),10 (11.8),60 (70.6)
Other: 115,10 (8.7),10 (8.7),- (-),15 (13.0),10 (8.7),80 (69.6)


##### Changes in 5 PRIMIS:2021 ethnicity

,asian,black,mixed,white,other,any
Latest Ethnicity- 5 PRIMIS:2021,,,,,,
Asian: 95,10 (10.5),10 (10.5),10 (10.5),- (-),- (-),70 (73.7)
Black: 95,15 (15.8),- (-),10 (10.5),10 (10.5),10 (10.5),65 (68.4)
Mixed: 90,10 (11.1),- (-),- (-),- (-),10 (11.1),65 (72.2)
White: 95,- (-),- (-),10 (10.5),10 (10.5),- (-),75 (78.9)
Other: 100,10 (10.0),10 (10.0),10 (10.0),10 (10.0),10 (10.0),60 (60.0)


#### 16 Group

In [375]:
for definition in definitions_16:
        lowerlist_16 = [x.lower() for x in (list(code_dict_16[definition].values()))]
        df_state_change = pd.read_csv(f'../output/{output_path_16}/simple_state_change_{definition}{suffix}_registered.csv').set_index(definition)
        df_state_change.columns = df_state_change.columns.str.replace(definition + "_", "")
        df_state_change.columns = df_state_change.columns.str.lower()
        #resort rows
        df_state_change = df_state_change.reindex(list(code_dict_16[definition].values()))
        df_state_change = df_state_change.reset_index()
        
        df_state_change[definition]=df_state_change[definition]+": " +df_state_change["n"].apply(lambda x: "{:,.0f}".format(x))
        df_state_change = df_state_change.set_index(definition)
        globals()[f'df_sc_pct_{definition}'] = df_state_change
        for item in lowerlist_16 + list(["any"]):
            df_state_change[item + "_pct"]= round(
                    (df_state_change[item].div(df_state_change["n"])) * 100, 1
                )
        
            df_state_change[item] = (
                    df_state_change[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_state_change[item + "_pct"].astype(str)
                    + ")"
                )
        df_state_change=df_state_change[lowerlist_16+ list(["any"])]
        df_state_change = df_state_change.replace("nan (nan)", "- (-)")
        df_state_change = df_state_change.reset_index()
        df_state_change = df_state_change.rename(definition_dict, axis='columns')
        df_state_change.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_state_change = df_state_change.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        df_state_change.columns = df_state_change.columns.str.replace("_", " ")
        # display(df_state_change)

In [376]:
display(Markdown(f"""
Patients whose latest recorded ethnicity were grouped as `Other Black` were most likely to have a discordant ethnicity recording ({'{:,.0f}'.format(df_sc_pct_ethnicity_new_16["any_pct"][6])}% SNOMED:2022, {'{:,.0f}'.format(df_sc_pct_ethnicity_16["any_pct"][6])}% CTV3:2020, {'{:,.0f}'.format(df_sc_pct_ethnicity_primis_16["any_pct"][6])}% PRIMIS:2021).
"""))


Patients whose latest recorded ethnicity were grouped as `Other Black` were most likely to have a discordant ethnicity recording (67% SNOMED:2022, 50% CTV3:2020, 67% PRIMIS:2021).


In [377]:
for definition in definitions_16:
        lowerlist_16 = [x.lower() for x in (list(code_dict_16[definition].values()))]
        df_state_change = pd.read_csv(f'../output/{output_path_16}/simple_state_change_{definition}{suffix}_registered.csv').set_index(definition)
        df_state_change.columns = df_state_change.columns.str.replace(definition + "_", "")
        df_state_change.columns = df_state_change.columns.str.lower()
        #resort rows
        df_state_change = df_state_change.reindex(list(code_dict_16[definition].values()))
        df_state_change = df_state_change.reset_index()
        
        df_state_change[definition]=df_state_change[definition]+": " +df_state_change["n"].apply(lambda x: "{:,.0f}".format(x))
        df_state_change = df_state_change.set_index(definition)
        globals()[f'df_sc_pct_{definition}'] = df_state_change
        for item in lowerlist_16 + list(["any"]):
            df_state_change[item + "_pct"]= round(
                    (df_state_change[item].div(df_state_change["n"])) * 100, 1
                )
        
            df_state_change[item] = (
                    df_state_change[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_state_change[item + "_pct"].astype(str)
                    + ")"
                )
        df_state_change=df_state_change[lowerlist_16+ list(["any"])]
        df_state_change = df_state_change.replace("nan (nan)", "- (-)")
        df_state_change = df_state_change.reset_index()
        df_state_change = df_state_change.rename(definition_dict, axis='columns')
        df_state_change.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_state_change = df_state_change.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        df_state_change.columns = df_state_change.columns.str.replace("_", " ")
        display(Markdown(f"##### Changes in {definition_dict[definition]} ethnicity"))
        display(df_state_change)
        print(" \n")

##### Changes in 16 SNOMED:2022 ethnicity

,indian,pakistani,bangladeshi,other asian,caribbean,african,other black,white and black caribbean,white and black african,white and asian,other mixed,white british,white irish,other white,chinese,any other ethnic group,any
Latest Ethnicity- 16 SNOMED:2022,,,,,,,,,,,,,,,,,
Indian: 25,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (80.0)
Pakistani: 35,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),25 (71.4)
Bangladeshi: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),15 (50.0)
Other_Asian: 25,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),15 (60.0)
Caribbean: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (66.7)
African: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (66.7)
Other_Black: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (66.7)
White_and_Black_Caribbean: 35,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),25 (71.4)
White_and_Black_African: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (66.7)


##### Changes in 16 CTV3:2020 ethnicity

,indian,pakistani,bangladeshi,other asian,caribbean,african,other black,white and black caribbean,white and black african,white and asian,other mixed,white british,white irish,other white,chinese,any other ethnic group,any
Latest Ethnicity- 16 CTV3:2020,,,,,,,,,,,,,,,,,
Indian: 45,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),10 (22.2),- (-),30 (66.7)
Pakistani: 25,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (80.0)
Bangladeshi: 20,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),15 (75.0)
Other_Asian: 35,- (-),- (-),- (-),- (-),- (-),- (-),10 (28.6),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),25 (71.4)
Caribbean: 25,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),15 (60.0)
African: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (66.7)
Other_Black: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),15 (50.0)
White_and_Black_Caribbean: 40,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),30 (75.0)
White_and_Black_African: 25,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),25 (100.0)


##### Changes in 16 PRIMIS:2021 ethnicity

,indian,pakistani,bangladeshi,other asian,caribbean,african,other black,white and black caribbean,white and black african,white and asian,other mixed,white british,white irish,other white,chinese,any other ethnic group,any
Latest Ethnicity- 16 PRIMIS:2021,,,,,,,,,,,,,,,,,
Indian: 40,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),30 (75.0)
Pakistani: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),15 (50.0)
Bangladeshi: 25,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),15 (60.0)
Other_Asian: 35,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (57.1)
Caribbean: 35,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (57.1)
African: 25,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (80.0)
Other_Black: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (66.7)
White_and_Black_Caribbean: 30,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),25 (83.3)
White_and_Black_African: 25,- (-),- (-),10 (40.0),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),20 (80.0)


<a id='frequent_cell'></a> 

### Comparison of 'Latest' and 'Most Frequent' coded ethnicity

#### 5 Group

In [378]:
from cmath import isnan


for definition in definitions_5:
        df_sum = pd.read_csv(f'../output/{output_path_5}/simple_latest_common_{definition}{suffix}_registered.csv').set_index(definition)
        # sort rows by category index
        df_sum.columns = df_sum.columns.str.replace(definition + "_", "")
        df_sum.columns = df_sum.columns.str.lower()
        df_sum = df_sum.reindex(list(code_dict_5[definition].values()))
        df_sum = df_sum[lowerlist_5]
        df_counts = pd.DataFrame(
            np.diagonal(df_sum),
            index=df_sum.index,
        #   columns=[f"matching (n={np.diagonal(df_sum).sum()})"],
        )

        df_sum2 = df_sum.copy(deep=True)
        np.fill_diagonal(df_sum2.values, 0)
        df_diag = pd.DataFrame(
            df_sum2.sum(axis=1),
        )
        df_out = df_counts.merge(df_diag, right_index=True, left_index=True)
        columns=round(df_out.sum()/df_out.sum(axis=1).sum()*100,1)
        globals()[f'df_col_{definition}'] = columns
        df_out.columns=[f"matching ({columns[0]}%)",f"not matching ({columns[1]}%)"]
        df_out = df_out.reset_index()
        df_out = df_out.rename(definition_dict, axis='columns')
        df_out.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_out = df_out.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        df_out = df_out.replace(np.nan, "-")
        # display(df_out)
        
        if code_dict_5 != "":
            lowerlist_5 = [x.lower() for x in (list(code_dict_5[definition].values()))]
            df_sum = df_sum[lowerlist_5]
        else:
            df_sum = df_sum.reindex(sorted(df_sum.columns), axis=1)

        # Combine count and percentage columns
        df_sum["population"]=df_sum.sum(axis = 1)
        globals()[f'df_sum_pct_{definition}'] = df_sum
        for item in lowerlist_5:
            df_sum[item + "_pct"]= round(
                    (df_sum[item].div(df_sum["population"])) * 100, 1
                )
            df_sum[item] = (
                    df_sum[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_sum[item + "_pct"].astype(str)
                    + ")"
                )
        df_sum = df_sum[lowerlist_5]
        df_sum = df_sum.reset_index()
        df_sum = df_sum.replace("nan (nan)", "- (-)")
        df_sum = df_sum.rename(definition_dict, axis='columns')
        df_sum.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_sum = df_sum.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        # display(df_sum)


In [379]:
display(Markdown(f"""
Overall {'{:,.0f}'.format(df_col_ethnicity_5[0])}% of the latest `5 group` ethnicity matched the most frequent `5 group` ethnicity for all codelists. {'{:,.0f}'.format(df_sum_pct_ethnicity_new_5["white_pct"][3])}% of those with the most recent ethnicity classified as `White` also had the most frequent ethnicity `White` for all three codelists. `Mixed` was the least concordant for all three codelists with {'{:,.0f}'.format(df_sum_pct_ethnicity_new_5["mixed_pct"][2])}% (SNOMED:2022 and CTV:2020) and {'{:,.0f}'.format(df_sum_pct_ethnicity_primis_5["mixed_pct"][2])}% (PRIMIS:2021) of those with the most recent ethnicity `Mixed` also had the most frequent ethnicity `Mixed`. Of those with latest ethnicity `Black` {'{:,.0f}'.format(df_sum_pct_ethnicity_new_5["white_pct"][1])}% also had the most frequent ethnicity `White`.
"""))


Overall 21% of the latest `5 group` ethnicity matched the most frequent `5 group` ethnicity for all codelists. 20% of those with the most recent ethnicity classified as `White` also had the most frequent ethnicity `White` for all three codelists. `Mixed` was the least concordant for all three codelists with nan% (SNOMED:2022 and CTV:2020) and nan% (PRIMIS:2021) of those with the most recent ethnicity `Mixed` also had the most frequent ethnicity `Mixed`. Of those with latest ethnicity `Black` nan% also had the most frequent ethnicity `White`.


In [380]:
for definition in definitions_5:
        df_sum = pd.read_csv(f'../output/{output_path_5}/simple_latest_common_{definition}{suffix}_registered.csv').set_index(definition)
        # sort rows by category index
        df_sum.columns = df_sum.columns.str.replace(definition + "_", "")
        df_sum.columns = df_sum.columns.str.lower()
        df_sum = df_sum.reindex(list(code_dict_5[definition].values()))
        df_sum = df_sum[lowerlist_5]
        df_counts = pd.DataFrame(
            np.diagonal(df_sum),
            index=df_sum.index,
        #   columns=[f"matching (n={np.diagonal(df_sum).sum()})"],
        )
        df_sum2 = df_sum.copy(deep=True)
        np.fill_diagonal(df_sum2.values, 0)
        df_diag = pd.DataFrame(
            df_sum2.sum(axis=1),
        )
        df_out = df_counts.merge(df_diag, right_index=True, left_index=True)
        columns=round(df_out.sum()/df_out.sum(axis=1).sum()*100,1)
        globals()[f'df_col_{definition}'] = columns
        df_out.columns=[f"matching ({columns[0]}%)",f"not matching ({columns[1]}%)"]
        df_out = df_out.reset_index()
        df_out = df_out.rename(definition_dict, axis='columns')
        df_out.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_out = df_out.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        df_out = df_out.replace(np.nan, "-")
        display(Markdown(f"##### 'Latest' and 'Most Frequent' {definition_dict[definition]} ethnicity"))
        display(df_out)
        print(" \n")
        
        if code_dict_5 != "":
            lowerlist_5 = [x.lower() for x in (list(code_dict_5[definition].values()))]
            df_sum = df_sum[lowerlist_5]
        else:
            df_sum = df_sum.reindex(sorted(df_sum.columns), axis=1)

        # Combine count and percentage columns
        df_sum["population"]=df_sum.sum(axis = 1)
        globals()[f'df_sum_pct_{definition}'] = df_sum
        for item in lowerlist_5:
            df_sum[item + "_pct"]= round(
                    (df_sum[item].div(df_sum["population"])) * 100, 1
                )
            df_sum[item] = (
                    df_sum[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_sum[item + "_pct"].astype(str)
                    + ")"
                )
        df_sum = df_sum[lowerlist_5]
        df_sum = df_sum.reset_index()
        df_sum = df_sum.replace("nan (nan)", "- (-)")
        df_sum = df_sum.rename(definition_dict, axis='columns')
        df_sum.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_sum = df_sum.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        display(Markdown(f"### 'Latest' and 'Most Frequent' {definition_dict[definition]} ethnicity expanded"))
        display(df_sum)
        print(" \n")


##### 'Latest' and 'Most Frequent' 5 SNOMED:2022 ethnicity

,matching (20.0%),not matching (80.0%)
Latest Ethnicity- 5 SNOMED:2022,,
Asian,-,0
Black,-,0
Mixed,-,0
White,20,80
Other,-,0


### 'Latest' and 'Most Frequent' 5 SNOMED:2022 ethnicity expanded

,asian,black,mixed,white,other
Latest Ethnicity- 5 SNOMED:2022,,,,,
Asian,- (-),- (-),- (-),- (-),- (-)
Black,- (-),- (-),- (-),- (-),- (-)
Mixed,- (-),- (-),- (-),- (-),- (-)
White,20 (20.0),20 (20.0),15 (15.0),20 (20.0),25 (25.0)
Other,- (-),- (-),- (-),- (-),- (-)


##### 'Latest' and 'Most Frequent' 5 CTV3:2020 ethnicity

...

,asian,black,mixed,white,other
Latest Ethnicity- 5 CTV3:2020,,,,,
Asian,10 (25.0),10 (25.0),- (-),10 (25.0),10 (25.0)
Black,- (-),10 (33.3),10 (33.3),- (-),10 (33.3)
Mixed,15 (27.3),10 (18.2),10 (18.2),10 (18.2),10 (18.2)
White,- (-),- (-),10 (33.3),10 (33.3),10 (33.3)
Other,10 (28.6),10 (28.6),- (-),15 (42.9),- (-)


##### 'Latest' and 'Most Frequent' 5 PRIMIS:2021 ethnicity

,matching (22.2%),not matching (77.8%)
Latest Ethnicity- 5 PRIMIS:2021,,
Asian,10,10
Black,-,45
Mixed,-,10
White,10,0
Other,10,40


### 'Latest' and 'Most Frequent' 5 PRIMIS:2021 ethnicity expanded

,asian,black,mixed,white,other
Latest Ethnicity- 5 PRIMIS:2021,,,,,
Asian,10 (50.0),10 (50.0),- (-),- (-),- (-)
Black,15 (33.3),- (-),10 (22.2),10 (22.2),10 (22.2)
Mixed,10 (100.0),- (-),- (-),- (-),- (-)
White,- (-),- (-),- (-),10 (100.0),- (-)
Other,10 (20.0),10 (20.0),10 (20.0),10 (20.0),10 (20.0)


#### 16 group

In [381]:
for definition in definitions_16:
        df_sum = pd.read_csv(f'../output/{output_path_16}/simple_latest_common_{definition}{suffix}_registered.csv').set_index(definition)    
        # sort rows by category index
        df_sum.columns = df_sum.columns.str.replace(definition + "_", "")
        df_sum.columns = df_sum.columns.str.lower()
        df_sum = df_sum.reindex(list(code_dict_16[definition].values()))
        df_sum = df_sum[lowerlist_16]
        df_counts = pd.DataFrame(
            np.diagonal(df_sum),
            index=df_sum.index,
        #   columns=[f"matching (n={np.diagonal(df_sum).sum()})"],
        )

        df_sum2 = df_sum.copy(deep=True)
        np.fill_diagonal(df_sum2.values, 0)
        df_diag = pd.DataFrame(
            df_sum2.sum(axis=1),
        )
        df_out = df_counts.merge(df_diag, right_index=True, left_index=True)
        columns=round(df_out.sum()/df_out.sum(axis=1).sum()*100,1)
        globals()[f'df_col_{definition}'] = columns
        df_out.columns=[f"matching ({columns[0]}%)",f"not matching ({columns[1]}%)"]
        df_out = df_out.reset_index()
        df_out = df_out.rename(definition_dict, axis='columns')
        df_out.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_out = df_out.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        df_out = df_out.replace(np.nan, "-")
        # display(df_out)
        
        if code_dict_16 != "":
            lowerlist_16 = [x.lower() for x in (list(code_dict_16[definition].values()))]
            df_sum = df_sum[lowerlist_16]
        else:
            df_sum = df_sum.reindex(sorted(df_sum.columns), axis=1)

        # Combine count and percentage columns
        df_sum["population"]=df_sum.sum(axis = 1)
        globals()[f'df_sum_pct_{definition}'] = df_sum
        for item in lowerlist_16:
            df_sum[item + "_pct"]= round(
                    (df_sum[item].div(df_sum["population"])) * 100, 1
                )
        
            df_sum[item] = (
                    df_sum[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_sum[item + "_pct"].astype(str)
                    + ")"
                )
        df_sum = df_sum[lowerlist_16]
        df_sum = df_sum.reset_index()
        df_sum = df_sum.replace("nan (nan)", "- (-)")
        df_sum = df_sum.rename(definition_dict, axis='columns')
        df_sum.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_sum = df_sum.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        df_sum.columns = df_sum.columns.str.replace("_", " ")
        
        
        # display(df_sum)


In [382]:
display(Markdown(f"""
Expanding to the `16 group` the percentage of latest ethnicity that match the most frequent ethnicity falls to {'{:,.0f}'.format(df_col_ethnicity_new_16[0])}% for both SNOMED:2022 and CTV3:2020 and {'{:,.0f}'.format(df_col_ethnicity_primis_16[0])}% for PRIMIS:2021. `White British` was the most concordant for both SNOMED:2022 and CTV3:2020 with {'{:,.0f}'.format(df_sum_pct_ethnicity_new_16["white_british_pct"][11])}% and {'{:,.0f}'.format(df_sum_pct_ethnicity_16["white_british_pct"][11])}%, respectively, of those with the most recent ethnicity classified as `White British` also had the most frequent ethnicity `White British`. For both SNOMED:2022 and CTV3:2020 `Other Black` was the least concordant, {'{:,.0f}'.format(df_sum_pct_ethnicity_new_16["other_black_pct"][6])}% of those with the most recent ethnicity `Other Black` also had the most frequent ethnicity `Other Black`.
"""))


Expanding to the `16 group` the percentage of latest ethnicity that match the most frequent ethnicity falls to nan% for both SNOMED:2022 and CTV3:2020 and nan% for PRIMIS:2021. `White British` was the most concordant for both SNOMED:2022 and CTV3:2020 with nan% and nan%, respectively, of those with the most recent ethnicity classified as `White British` also had the most frequent ethnicity `White British`. For both SNOMED:2022 and CTV3:2020 `Other Black` was the least concordant, nan% of those with the most recent ethnicity `Other Black` also had the most frequent ethnicity `Other Black`.


In [383]:
for definition in definitions_16:
        df_sum = pd.read_csv(f'../output/{output_path_16}/simple_latest_common_{definition}{suffix}_registered.csv').set_index(definition)    
        # sort rows by category index
        df_sum.columns = df_sum.columns.str.replace(definition + "_", "")
        df_sum.columns = df_sum.columns.str.lower()
        df_sum = df_sum.reindex(list(code_dict_16[definition].values()))
        df_sum = df_sum[lowerlist_16]
        df_counts = pd.DataFrame(
            np.diagonal(df_sum),
            index=df_sum.index,
        #   columns=[f"matching (n={np.diagonal(df_sum).sum()})"],
        )

        df_sum2 = df_sum.copy(deep=True)
        np.fill_diagonal(df_sum2.values, 0)
        df_diag = pd.DataFrame(
            df_sum2.sum(axis=1),
        )
        df_out = df_counts.merge(df_diag, right_index=True, left_index=True)
        columns=round(df_out.sum()/df_out.sum(axis=1).sum()*100,1)
        globals()[f'df_col_{definition}'] = columns
        df_out.columns=[f"matching ({columns[0]}%)",f"not matching ({columns[1]}%)"]
        df_out = df_out.reset_index()
        df_out = df_out.rename(definition_dict, axis='columns')
        df_out.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_out = df_out.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        df_out = df_out.replace(np.nan, "-")

        display(Markdown(f"##### 'Latest' and 'Most Frequent' {definition_dict[definition]} ethnicity"))
        display(df_out)
        print(" \n")

        if code_dict_16 != "":
            lowerlist_16 = [x.lower() for x in (list(code_dict_16[definition].values()))]
            df_sum = df_sum[lowerlist_16]
        else:
            df_sum = df_sum.reindex(sorted(df_sum.columns), axis=1)

        # Combine count and percentage columns
        df_sum["population"]=df_sum.sum(axis = 1)
        globals()[f'df_sum_pct_{definition}'] = df_sum
        for item in lowerlist_16:
            df_sum[item + "_pct"]= round(
                    (df_sum[item].div(df_sum["population"])) * 100, 1
                )
        
            df_sum[item] = (
                    df_sum[item].apply(lambda x: "{:,.0f}".format(x))
                    + " ("
                    + df_sum[item + "_pct"].astype(str)
                    + ")"
                )
        df_sum = df_sum[lowerlist_16]
        df_sum = df_sum.reset_index()
        df_sum = df_sum.replace("nan (nan)", "- (-)")
        df_sum = df_sum.rename(definition_dict, axis='columns')
        df_sum.rename(columns={f'{definition_dict[definition]}':f'Latest Ethnicity-\n{definition_dict[definition]}'}, inplace=True)
        df_sum = df_sum.set_index(f'Latest Ethnicity-\n{definition_dict[definition]}')
        df_sum.columns = df_sum.columns.str.replace("_", " ")
        
        display(Markdown(f"### 'Latest' and 'Most Frequent' {definition_dict[definition]} ethnicity expanded"))
        display(df_sum)
        print(" \n")

##### 'Latest' and 'Most Frequent' 16 SNOMED:2022 ethnicity

,matching (nan%),not matching (nan%)
Latest Ethnicity- 16 SNOMED:2022,,
Indian,-,0
Pakistani,-,0
Bangladeshi,-,0
Other_Asian,-,0
Caribbean,-,0
African,-,0
Other_Black,-,0
White_and_Black_Caribbean,-,0
White_and_Black_African,-,0


### 'Latest' and 'Most Frequent' 16 SNOMED:2022 ethnicity expanded

,indian,pakistani,bangladeshi,other asian,caribbean,african,other black,white and black caribbean,white and black african,white and asian,other mixed,white british,white irish,other white,chinese,any other ethnic group
Latest Ethnicity- 16 SNOMED:2022,,,,,,,,,,,,,,,,
Indian,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Pakistani,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Bangladeshi,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Other_Asian,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Caribbean,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
African,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Other_Black,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
White_and_Black_Caribbean,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
White_and_Black_African,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)


##### 'Latest' and 'Most Frequent' 16 CTV3:2020 ethnicity

...

,indian,pakistani,bangladeshi,other asian,caribbean,african,other black,white and black caribbean,white and black african,white and asian,other mixed,white british,white irish,other white,chinese,any other ethnic group
Latest Ethnicity- 16 CTV3:2020,,,,,,,,,,,,,,,,
Indian,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),10 (100.0),- (-)
Pakistani,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Bangladeshi,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Other_Asian,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Caribbean,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
African,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Other_Black,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
White_and_Black_Caribbean,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
White_and_Black_African,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)


##### 'Latest' and 'Most Frequent' 16 PRIMIS:2021 ethnicity

,matching (nan%),not matching (nan%)
Latest Ethnicity- 16 PRIMIS:2021,,
Indian,-,0
Pakistani,-,0
Bangladeshi,-,0
Other_Asian,-,0
Caribbean,-,0
African,-,0
Other_Black,-,0
White_and_Black_Caribbean,-,0
White_and_Black_African,-,0


### 'Latest' and 'Most Frequent' 16 PRIMIS:2021 ethnicity expanded

,indian,pakistani,bangladeshi,other asian,caribbean,african,other black,white and black caribbean,white and black african,white and asian,other mixed,white british,white irish,other white,chinese,any other ethnic group
Latest Ethnicity- 16 PRIMIS:2021,,,,,,,,,,,,,,,,
Indian,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Pakistani,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Bangladeshi,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Other_Asian,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Caribbean,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
African,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
Other_Black,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
White_and_Black_Caribbean,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)
White_and_Black_African,- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-),- (-)


### Discussion

In [384]:
display(Markdown(f"""
This study has shown that primary care ethnicity data made available via OpenSAFELY is complete for around three quarters of all patients. However, recording ethnicity is not straightforward. Indeed, despite often being used as a key variable to describe health, the idea of "ethnicity" has been disputed. Self-identified ethnicity is not a fixed concept and evolving socio-cultural trends could contribute to changes in a person's self-identified ethnic group, particularly for those with mixed heritage. It is therefore perhaps not surprising to see lower levels of concordance between latest ethnicity and most common ethnicity in those with latest ethnicity grouped as `mixed`. 

In OpenSAFELY ethnicity in primary care was originally grouped using the CTV3:2020 codelist. However, this codelist does not strictly follow the grouping of the 2001 Census, which is the NHS standard for ethnicity. The common practice of supplementing CTV3:2020 coded ethnicity with SUS data has been shown to provide ethnicity data for [90.6% of patients](https://wellcomeopenresearch.org/articles/7-191/v1), however could lead to inconsistent classification as SUS data follow the 2001 census groups. 

We believe that the SNOMED:2022 codelist provides a more consistent representation of ethnicity as defined by the 2001 census groups and should be the preferred codelist for primary care ethnicity. 
"""))


This study has shown that primary care ethnicity data made available via OpenSAFELY is complete for around three quarters of all patients. However, recording ethnicity is not straightforward. Indeed, despite often being used as a key variable to describe health, the idea of "ethnicity" has been disputed. Self-identified ethnicity is not a fixed concept and evolving socio-cultural trends could contribute to changes in a person's self-identified ethnic group, particularly for those with mixed heritage. It is therefore perhaps not surprising to see lower levels of concordance between latest ethnicity and most common ethnicity in those with latest ethnicity grouped as `mixed`. 

In OpenSAFELY ethnicity in primary care was originally grouped using the CTV3:2020 codelist. However, this codelist does not strictly follow the grouping of the 2001 Census, which is the NHS standard for ethnicity. The common practice of supplementing CTV3:2020 coded ethnicity with SUS data has been shown to provide ethnicity data for [90.6% of patients](https://wellcomeopenresearch.org/articles/7-191/v1), however could lead to inconsistent classification as SUS data follow the 2001 census groups. 

We believe that the SNOMED:2022 codelist provides a more consistent representation of ethnicity as defined by the 2001 census groups and should be the preferred codelist for primary care ethnicity. 


### Limitations

In [385]:
display(Markdown(f"""
It is common for OpenSAFELY studies to supplement the primary care recorded ethnicity, where missing, with ethnicity data from the Secondary Uses Service (SUS). This study has focussed solely on the primary care recorded ethnicity. Due to the way that non-native data, such as GP2GP data and historical data, are imported into TPP the date of ethnicity recorded is not always available therefore chronology is unreliable for ethnicity data. 
"""))


It is common for OpenSAFELY studies to supplement the primary care recorded ethnicity, where missing, with ethnicity data from the Secondary Uses Service (SUS). This study has focussed solely on the primary care recorded ethnicity. Due to the way that non-native data, such as GP2GP data and historical data, are imported into TPP the date of ethnicity recorded is not always available therefore chronology is unreliable for ethnicity data. 


### Conclusion

In [386]:
display(Markdown(f"""
This report describes existing methods to derive primary care ethnicity in OpenSAFELY-TPP and suggests the adoption of the SNOMED:2022 codelist as the new standard method. It is a living document that can be periodically re-run to evaluate the most current best practices for research. If you have improvements or forks, please contact the OpenSAFELY data team.
"""))


This report describes existing methods to derive primary care ethnicity in OpenSAFELY-TPP and suggests the adoption of the SNOMED:2022 codelist as the new standard method. It is a living document that can be periodically re-run to evaluate the most current best practices for research. If you have improvements or forks, please contact the OpenSAFELY data team.


In [387]:
from datetime import date, timedelta
# get data extraction date
extract_date = pd.to_datetime(os.path.getmtime(f"../output/{output_path_16}/simple_patient_counts_registered.csv"), unit='s')
# get notebook run date
run_date = date.today()

display(Markdown(f"""
## Technical details

This notebook was run on {run_date.strftime('%Y-%m-%d')}. The information below is based on data extracted from the OpenSAFELY-TPP database on {extract_date.strftime('%Y-%m-%d')}.

If a clinical code appears in the primary care record on multiple dates, the latest date is used. 


Only patients registered at their practice on January 1 2022 are included.

"""))


## Technical details

This notebook was run on 2023-02-16. The information below is based on data extracted from the OpenSAFELY-TPP database on 2023-02-15.

If a clinical code appears in the primary care record on multiple dates, the latest date is used. 


Only patients registered at their practice on January 1 2022 are included.

